Searching for VIIRS L2 fire data (swaths and text files) in the cloud and checking which versions are available

In [ ]:
import xarray as xr
import earthaccess
from earthaccess import Auth, Store, DataCollections, DataGranules
import datetime as dt
from pprint import pprint
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#optional for plotting
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [ ]:
EXTENT = [-119.55,36.95,-118.9,37.7] #creek fire

START = '2020-09-06'
END = '2020-09-08'
#dates converted to UTC time; end date is exclusive

SATELLITE = 'SNPP' #'SNPP' or 'NOAA20'

if SATELLITE=='SNPP': products = ['VNP03IMG','VNP02IMG','VNP14IMG']
elif SATELLITE=='NOAA20': products = ['VJ103IMG','VJ102IMG','VJ114IMG'] 

In [ ]:
#first get the L1 geolocation and science data via direct S3 access
#may require restarting the kernel if permission errors

files = {}

#geolocation 03IMG
results = earthaccess.search_data(
    short_name=products[0],
    bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
    temporal=(START, END),
    count=10
)
files[products[0]] = earthaccess.open(results)

#science data 02IMG
results = earthaccess.search_data(
    short_name=products[1],
    bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
    temporal=(START, END),
    count=10
)
files[products[1]] = earthaccess.open(results)

files

In [ ]:
#next get the Level2 files - need to login to LAADS
#was getting permission errors trying to do both with the same method - likely related to region storage

auth = Auth() #need to reauthenticate every hour or so :(
#auth.login(strategy="interactive", persist=True) #RUN THIS THE FIRST TIME
auth.login(strategy="netrc") #read credentials from previously saved ~/.netrc file

store = Store(auth)
fs = store.get_s3fs_session('LAADS') #daac or provider name

In [ ]:
product = products[2]

Query = DataGranules().short_name(product).bounding_box(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]).temporal(START,END)

print(Query.hits(), 'hits')
cloud_granules = Query.get(800) #first 800 results
print('cloud hosted', cloud_granules[0].cloud_hosted)

s3_links = {}
s3_links[product] = []
for granule in cloud_granules:
    s3_links[product].extend(granule.data_links(access="in-region"))
s3_links[product] = sorted(s3_links[product]) 
files[product] = store.open(s3_links[product], provider="LAADS")

print(product)
pprint(files)

In [ ]:
#test open one 03IMG geolocation (optional)
xr.open_dataset(files[products[0]][0], engine='h5netcdf', group='geolocation_data')

In [ ]:
#test open one 14IMG level 2 (optional)
xr.open_dataset(files[products[2]][0], phony_dims='sort')

In [ ]:
files[products[0]][0].path

In [ ]:
files[products[2]][1].path

In [ ]:
import matplotlib as mpl
from matplotlib.colors import ListedColormap

mask_colors = [mpl.colormaps['tab10'](c) for c in [4,6,5,0,9,2,7,8,1,3]] #fire mask colors
dets_colors = ['white']*7 + ['black']*3                                  #black and white version

cmp1 = ListedColormap(mask_colors)
cmp2 = ListedColormap(dets_colors)

In [ ]:
all_dets = pd.DataFrame()

for i in range(len(files[products[0]])): #VNP03IMG or VJ103IMG
    timestamp = files[products[0]][i].path.split('.')[-5:-3]
    print(timestamp)
    year = timestamp[0][1:5]
    day = timestamp[0][5:8]
    time = timestamp[1]
    date = dt.datetime.strptime(year+day, '%Y%j').strftime('%b %d') 
    acq_datetime = dt.datetime.strptime(year+day+time[:2]+time[2:], '%Y%j%H%M').strftime('%Y-%m-%d %H:%M:00 +00:00') 
    daytime = int(time) > 1500 #depends on timezone

    try:
        #open 03IMG geolocation
        geo = xr.open_dataset(files[products[0]][i], engine='h5netcdf', group='geolocation_data')
        lon = geo['longitude'][:]
        lat = geo['latitude'][:]
        _, j = np.indices(geo.longitude.shape) #line and sample
        
        scene = (lon > EXTENT[0]) & (lon < EXTENT[2]) & (lat > EXTENT[1]) & (lat < EXTENT[3])

        #crop down the datasets for memory 
        indices = np.where(scene)
        x0 = indices[0].min()
        x1 = indices[0].max()
        y0 = indices[1].min()
        y1 = indices[1].max()

        lon = lon[x0:x1, y0:y1]
        lat = lat[x0:x1, y0:y1]
        j = j[x0:x1, y0:y1]
        
        #open 02IMG science data, i4 band
        data = xr.open_dataset(files[products[1]][i], engine='h5netcdf', group='observation_data')
        data = data.sel(number_of_lines=slice(x0,x1), number_of_pixels=slice(y0,y1))
        
        i4 = data['I04'] #xarray already encodes the scale factor and offset
        scale = data.I04.encoding['scale_factor']
        offset = data.I04.encoding['add_offset']
        i4 = (i4[:,:] - offset) / scale #return to raw values to use lookup table to temperature
        i4 = i4.astype(int)
        i4_bt = data['I04_brightness_temperature_lut'][:]
        i4_bt = i4_bt[i4]
        
        #get VNP14IMG
        data = xr.open_dataset(files[products[2]][i], phony_dims='sort')
        data = data.sel(phony_dim_1=slice(x0,x1), phony_dim_2=slice(y0,y1))
        daynight = data.DayNightFlag #string Day or Night
        
        qa = data.variables['algorithm QA'][:]
        fire = data.variables['fire mask'][:]  
        fires = (fire>6).values

    except:
        print('error with file',timestamp)
        stop
        continue
    
    #look at QA flags data next over entire scene
    values, counts = np.unique(qa, return_counts=True)

    table = pd.DataFrame(index = values, columns=range(22,-1,-1)) #[22,21,...0]
    for i1 in table.index:
        b = np.binary_repr(i1, width=23)
        b = [int(s) for s in b]
        table.loc[i1, :] = b
    
    #report back all the pixels that have an 8 or 10 ~ background or candidate fires
    keep = table[(table.loc[:,8]==1) | (table.loc[:,10]==1)].index
    keep = (np.isin(qa[:], keep) | (fires))  #"fires" because some low conf are Test 16 pixel saturation
    
    
    #build pandas table for exporting, following VIIRS L2 columns
    i_dets = pd.DataFrame()
    i_dets['longitude'] = list(lon.values[keep])
    i_dets['latitude'] = list(lat.values[keep])
    i_dets['fire_mask'] = list(fire.values[keep])
    i_dets['daynight'] = daynight[0]
    i_dets['confidence'] = i_dets.fire_mask
    i_dets.confidence = i_dets.confidence.replace({0:'x', 1:'x', 2:'x', 3:'x', 4:'x', 5:'x', 6:'x', 7:'l', 8:'n', 9:'h'})
    i_dets['acq_date'] = date
    i_dets['acq_time'] = time
    i_dets['acq_datetime'] = acq_datetime
    i_dets['j'] = list(j[keep]) #sample number for pixel size lookup
   
    #crop down to defined extent
    i_dets = i_dets[(i_dets.longitude > EXTENT[0]) & (i_dets.longitude < EXTENT[2]) & (i_dets.latitude > EXTENT[1]) & (i_dets.latitude < EXTENT[3])]
    
    #FIGURE ----------------
   
    fig, ((ax,ax2,ax3,ax4),(ax5,ax6,ax7,ax8)) = plt.subplots(2,4, gridspec_kw={'width_ratios':[3,3,3,1], 'height_ratios':[6,1]}, constrained_layout=True, subplot_kw={'projection':ccrs.Miller()}, figsize=(12,8))

    #Level 1 imagery
    ax.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
    plot = ax.pcolormesh(lon, lat, i4_bt, vmin=250, vmax=360, cmap='plasma', transform=ccrs.PlateCarree())
    cbar = plt.colorbar(plot, orientation='horizontal', shrink=0.6, pad=-2.2, extend='both', ax=ax5)
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label('I4 brightness temperature (K)', size=12)

    #Level 1 imagery plus detections
    ax2.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
    plot = ax2.pcolormesh(lon, lat, i4_bt, vmin=250, vmax=360, cmap='plasma', transform=ccrs.PlateCarree())
    cbar = plt.colorbar(plot, orientation='horizontal', shrink=0.6, pad=-2.2, extend='both', ax=ax6)
    cbar.ax.tick_params(labelsize=12)
    cbar.set_label('I4 brightness temperature (K)', size=12)

    ax2.scatter(i_dets.longitude, i_dets.latitude, c=cmp2(i_dets['fire_mask'].astype(int)), s=0.5, transform=ccrs.Geodetic())
    ax2.text(0.2, 0.9, 'Known fire pixels', c='black', transform = ax2.transAxes, fontsize=12)
    ax2.text(0.2, 0.85, 'Candidate fire pixels', c='white', transform = ax2.transAxes, fontsize=12)
    ax2.set_title(f'{SATELLITE} {date} {time}h UTC')

    #Level 2 fire mask
    ax3.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
    plot = ax3.pcolormesh(lon, lat, fire, vmin=0, vmax=10, cmap=cmp1, transform=ccrs.PlateCarree())

    #Level 2 fire mask legend
    cbar = plt.colorbar(plot, orientation='vertical', shrink=0.8, pad=-1, ax=ax4)

    labels = ['0 not-processed', '1 bowtie', '2 glint', '3 water','4 clouds',
          '5 clear land','6 unclassified fire pixel','7 low confidence fire pixel',
          '8 nominal confidence fire pixel','9 high confidence fire pixel']
    cbar.ax.set_yticks(np.arange(len(labels))+0.5)
    cbar.ax.set_yticklabels(labels) 
    cbar.ax.tick_params(labelsize=12)
    #cbar.set_label('Fire mask', size=13)
    #ax3.set_title('Fire mask')

    ax4.axis('off')
    ax5.axis('off')
    ax6.axis('off')
    ax7.axis('off')
    ax8.axis('off')
    plt.savefig(f'/projects/shared-buckets/coffield/figures/creek_{timestamp[0]}-{timestamp[1]}_{SATELLITE}.png', dpi=150, bbox_inches='tight')

    all_dets = pd.concat([all_dets, i_dets])
    
#save csv with filename as the timestamp range
#all_dets.to_csv(f'/projects/shared-buckets/coffield/??PATH??{timestamp[0]}_{SATELLITE}.csv', index=False)

print('done')

<h3>Check each QA bit

In [ ]:
for i in range(6,len(files[products[0]])): #VNP03IMG or VJ103IMG
    timestamp = files[products[0]][i].path.split('.')[-5:-3]
    print(timestamp)
    year = timestamp[0][1:5]
    day = timestamp[0][5:8]
    time = timestamp[1]
    date = dt.datetime.strptime(year+day, '%Y%j').strftime('%b %d') 
    acq_datetime = dt.datetime.strptime(year+day+time[:2]+time[2:], '%Y%j%H%M').strftime('%Y-%m-%d %H:%M:00 +00:00') 
    daytime = int(time) > 1500 #depends on timezone

    try:
        #open VNP03IMG geolocation
        geo = xr.open_dataset(files[products[0]][i], engine='h5netcdf', group='geolocation_data')
        lon = geo['longitude'][:]
        lat = geo['latitude'][:]
        
        scene = (lon > EXTENT[0]) & (lon < EXTENT[2]) & (lat > EXTENT[1]) & (lat < EXTENT[3])

        #crop down the datasets for memory 
        indices = np.where(scene)
        x0 = indices[0].min()
        x1 = indices[0].max()
        y0 = indices[1].min()
        y1 = indices[1].max()

        lon = lon[x0:x1, y0:y1]
        lat = lat[x0:x1, y0:y1]
        
        #get VNP02IMG science data, i1-i5 bands
        data = xr.open_dataset(files[products[1]][i], phony_dims='sort')
        data = data.sel(phony_dim_1=slice(x0,x1), phony_dim_2=slice(y0,y1))
        
        qa = data.variables['algorithm QA'][:]
        fire = data.variables['fire mask'][:]  

    except:
        print('error with file or does not exist',timestamp)
        stop
        continue
    
    #look at QA flags data next over entire scene
    values, counts = np.unique(qa, return_counts=True)

    table = pd.DataFrame(index = values, columns=range(22,-1,-1)) #[22,21,...0]
    for i1 in table.index:
        b = np.binary_repr(i1, width=23)
        b = [int(s) for s in b]
        table.loc[i1, :] = b
    table['count'] = counts
    
    #eights += table[(table.loc[:,8] == 1)]['count'].sum()
    #tens += table[(table.loc[:,10] == 1)]['count'].sum()
    #noaa20_sums += table.iloc[:,:-1].multiply(table['count'], axis="index").sum()
    #print('---------', table[(table.loc[:,8] == 1)]['count'].sum(), table[(table.loc[:,10] == 1)]['count'].sum())
    
    '''
    #create new dataset with bands for each QA bit yes/no
    ds = xr.Dataset(
        data_vars=dict(
            qa=(["x", "y"], qa)
        ),
        coords=dict(
            lon=(["x", "y"], lon),
            lat=(["x", "y"], lat),
        ),
        attrs=dict(description="quality assurance bits"),
    )
    ds = ds.where((ds.lon > extent[0]) & (ds.lon < extent[1]) & (ds.lat > extent[2]) & (ds.lat < extent[3]))
    
    for i in range(2):
        ds = ds.assign(name = lambda ds: ds.qa * 0)
        ds = ds.rename({'name':'bit'+str(i)})
    '''
    
    bits = np.zeros((lon.shape[0], lon.shape[1], 23))
    
    for i in table.index[1:]: #skip the first row (zero)
        locs = np.where(qa==i)
        
        b = np.binary_repr(i, width=23)[::-1]
        bit_locs = [j for j in range(len(b)) if b[j]=='1']
        
        for bit in bit_locs:
            bits[locs[0],locs[1],bit] = 1
        
        del locs, b, bit_locs
    
    
    #figure: map of one QA bit
    fig = plt.figure(figsize=(19,15))

    for b in range(bits.shape[2]):
        ax = plt.subplot(4,6,b+1, projection = ccrs.Miller())

        #left panel map
        ax.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
        ax.pcolormesh(lon, lat, bits[:,:,b], vmin=0, vmax=1, transform=ccrs.PlateCarree())
        ax.set_title('Bit {}'.format(b))

    plt.suptitle(f'{SATELLITE} {date} {year} {time}h', fontsize=18, weight='bold')
    plt.tight_layout()
    #plt.savefig('/discover/nobackup/scoffie1/figures/bits/noaa20-{}-{}-{}.png'.format(day, year, time), dpi=150)
    #plt.close()
    stop
    
    
    '''    
    #build pandas table for exporting, following VIIRS L2 columns
    i_dets = pd.DataFrame() #copy of master table just for this swath
    i_dets['longitude'] = list(np.array(lon)[fires])
    i_dets['latitude'] = list(np.array(lat)[fires])
    i_dets['acq_date'] = dt.datetime.strptime(year+day, '%Y%j').strftime('%Y/%m/%d') 
    i_dets['acq_time'] = time
    i_dets['acq_datetime'] = acq_datetime
   
    #crop down to defined extent
    i_dets = i_dets[(i_dets.longitude > EXTENT[0]) & (i_dets.longitude < EXTENT[2]) & (i_dets.latitude > EXTENT[1]) & (i_dets.latitude < EXTENT[3])]
    '''
    #FIGURE ----------------
    fig = plt.figure(figsize=(11,8))

    ax = fig.add_subplot(121, projection=ccrs.Miller())
    ax.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
    plot = ax.pcolormesh(lon, lat, fire, vmin=0, vmax=10, cmap='plasma', transform=ccrs.PlateCarree())
    cbar = plt.colorbar(plot, orientation='horizontal', shrink=0.6, pad=0.01, extend='both', ax=ax)
    cbar.ax.tick_params(labelsize=13)
    cbar.set_label('I4 brightness temperature (K)', size=13)
    ax.set_title(f'{SATELLITE} {date} {time}h UTC') 
    stop         
    ax2 = fig.add_subplot(122, projection=ccrs.Miller())
    ax2.set_extent([EXTENT[0],EXTENT[2],EXTENT[1],EXTENT[3]])
    plot = ax2.pcolormesh(lon, lat, i4_bt, vmin=250, vmax=360, cmap='plasma', transform=ccrs.PlateCarree())
    cbar = plt.colorbar(plot, orientation='horizontal', shrink=0.6, pad=0.01, extend='both', ax=ax2)
    cbar.ax.tick_params(labelsize=13)
    cbar.set_label('I4 brightness temperature (K)', size=13)
    ax2.scatter(i_dets.longitude, i_dets.latitude, c='1', s=1, transform=ccrs.Geodetic())
    ax2.text(0.35, 0.9, 'Potential fire pixels', c='white', transform = ax2.transAxes)
    
    #plt.savefig(f'/projects/shared-buckets/coffield/??PATH??{timestamp[0]}-{timestamp[1]}_{SATELLITE}.png', dpi=150)

    all_dets = pd.concat([all_dets, i_dets])
    
#save csv with filename as the timestamp range
#all_dets.to_csv(f'/projects/shared-buckets/coffield/??PATH??{timestamp[0]}_{SATELLITE}.csv', index=False)

print('done')

<h3>Old code / debugging

Issue: cannot earthaccess.open on both VNP03IMG and VNP14IMG consecutively. Requires restarting the kernel and doing one or the other. Current workaround: download the VNP14IMGs, then restart and stream the VNP03IMG

In [ ]:
#run once then restart kernel
results = earthaccess.search_data(
    short_name='VNP14IMG',
    bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
    temporal=(START, END),
    count=10
)
#files = earthaccess.open(results)
files = earthaccess.download(results, '/projects/shared-buckets/coffield/viirs/VNP14IMG')
files

In [ ]:
files = {}

results = earthaccess.search_data(
    short_name='VNP03IMG',
    bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
    temporal=(START, END),
    count=10
)

files[product] = earthaccess.open(results)

files

In [ ]:
files = {}
for product in products:
    #query for granules - by bounding box or point
    results = earthaccess.search_data(
        short_name=product,
        bounding_box=(EXTENT[0],EXTENT[1],EXTENT[2],EXTENT[3]),
        temporal=(START, END),
        count=10
    )

    files[product] = earthaccess.open(results)
    
files

In [ ]:
import boto3
boto3.client('s3').meta.region_name

In [ ]:
files['VNP14IMG'][0]

In [ ]:
foo = xr.open_dataset(files[0]), phony_dims='sort')
foo

In [ ]:
plt.imshow(foo['fire mask'])

In [ ]:
#AGAIN FOR SNPP (new) 

#count QA flags for each swath

root = '/css/viirs/data/'

with open('/home/scoffie1/links/creek_VJ103IMG_sep2020_day.txt', 'r') as links: # VJ103IMG_082020_amazon.txt
    files = links.readlines()

#files = os.listdir(root + 'Level1/VJ103IMG.trimmed/2020/245') #ANYWHERE IN THE WORLD
files = os.listdir('/discover/nobackup/scoffie1/VJ114IMG')
files = [f for f in files if f[-2:]=='nc']

files = ['VNP03IMG.A2020251.2042.002.2021125052211.nc']
files = ['VNP03IMG.A2020251.0924']
files = sorted(files)
print(files)

values_noaa20_mask = np.array([])
eights = 0
tens = 0
noaa20_sums = 0

for file in files:
    day = file.split('.')[1][-3:]
    year = file.split('.')[1][1:5]
    date = dt.datetime.strptime(year+day, '%Y%j').strftime('%b %d') 
    time = file.split('.')[2] #time kept as a string - may affect plot titles
    print(day, time)

    try:
        #get VJ103 geolocation
        matches = os.listdir(root + 'Level1/VNP03IMG.trimmed/2020/' + day)
        matches = [match for match in matches if match[18:22] == time]
        geo = netCDF4.Dataset(root + 'Level1/VNP03IMG.trimmed/2020/' + day + '/' + matches[0])
        lon = geo.groups['geolocation_data']['longitude'][:]
        lat = geo.groups['geolocation_data']['latitude'][:]

        #get VJ114 fire mask + qa
        matches = os.listdir(root + 'Level2/VNP14IMG/2020/' + day)
        matches = [match for match in matches if match[18:22] == time]
        data = netCDF4.Dataset(root + 'Level2/VNP14IMG/2020/' + day + '/' + matches[0])
        #data = netCDF4.Dataset('/home/scoffie1/VNP14IMG/VNP14IMG.A2020251.2042.002.2023008022048.nc') #COLLECTION2
        data = netCDF4.Dataset('/home/scoffie1/VNP14IMG/VNP14IMG.A2020251.0924.002.2023008022055.nc') #COLLECTION2
        
        qa = data.variables['algorithm QA']
        fire = data.variables['fire mask']
    
    except:
        print('error with file',file)
        continue

    scene = (lon > extent[0]) & (lon < extent[1]) & (lat > extent[2]) & (lat < extent[3])

    #fire mask for entire scene flags data next over entire scene
    scene_mask = np.array(fire)[scene] 
    
    values_noaa20_mask = np.concatenate([values_noaa20_mask, scene_mask])
    
    #look at QA flags data next over entire scene
    scene_qa = np.array(qa)[scene] #[uncs][fires] #can play with subsetting
    values, counts = np.unique(scene_qa, return_counts=True)

    table = pd.DataFrame(index = values, columns=range(22,-1,-1)) #[22,21,...0]
    for i1 in table.index:
        b = np.binary_repr(i1, width=23)
        b = [int(s) for s in b]
        table.loc[i1, :] = b
    table['count'] = counts
    
    eights += table[(table.loc[:,8] == 1)]['count'].sum()
    tens += table[(table.loc[:,10] == 1)]['count'].sum()
    noaa20_sums += table.iloc[:,:-1].multiply(table['count'], axis="index").sum()
    print('---------', table[(table.loc[:,8] == 1)]['count'].sum(), table[(table.loc[:,10] == 1)]['count'].sum())
    
    '''
    #create new dataset with bands for each QA bit yes/no
    ds = xr.Dataset(
        data_vars=dict(
            qa=(["x", "y"], qa)
        ),
        coords=dict(
            lon=(["x", "y"], lon),
            lat=(["x", "y"], lat),
        ),
        attrs=dict(description="quality assurance bits"),
    )
    ds = ds.where((ds.lon > extent[0]) & (ds.lon < extent[1]) & (ds.lat > extent[2]) & (ds.lat < extent[3]))
    
        
    for i in range(2):
        ds = ds.assign(name = lambda ds: ds.qa * 0)
        ds = ds.rename({'name':'bit'+str(i)})
    '''
    
    bits = np.zeros((lon.shape[0], lon.shape[1], 23))
    qa = np.ma.masked_where(~scene, qa)
    
    for i in table.index[1:]: #skip the first row (zero)
        locs = np.where(qa==i)
        
        b = np.binary_repr(i, width=23)[::-1]
        bit_locs = [j for j in range(len(b)) if b[j]=='1']
        
        for bit in bit_locs:
            bits[locs[0],locs[1],bit] = 1
        
        del locs, b, bit_locs
    
    
    #four corners for cropping
    corners = np.where(~qa.mask)
    if len(corners[0]) > 0: #if there is any data in the scene
        x0 = corners[0].min()
        x1 = corners[0].max()
        y0 = corners[1].min()
        y1 = corners[1].max()


        #figure: map of one QA bit
        fig = plt.figure(figsize=(19,15))

        for b in range(bits.shape[2]):
            ax = plt.subplot(4,6,b+1, projection = ccrs.Miller())

            #left panel map
            ax.set_extent(extent)
            ax.pcolormesh(lon[x0:x1, y0:y1], lat[x0:x1, y0:y1], bits[x0:x1, y0:y1][:,:,b], vmin=0, vmax=1, transform=ccrs.PlateCarree())
            ax.set_title('Bit {}'.format(b))

        plt.suptitle('SNPP {} {} {}'.format(date, year, time), fontsize=18, weight='bold')
        plt.tight_layout()
        #plt.savefig('/discover/nobackup/scoffie1/figures/bits/noaa20-{}-{}-{}.png'.format(day, year, time), dpi=150)
        #plt.close()
    